In [18]:
#라이브러리 로딩
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import urllib.request
import urllib
import time
import pandas as pd
import os
import math

# 키워드 입력 받기
print("*" * 80)
print("쿠팡 사이트의 식품 카테고리 Best Seller 상품 정보 추출하기")
print("*" * 80)

cnt = int(input('1. 크롤링 할 건수는 몇 건입니까?: '))
page_cnt = math.ceil(cnt/60)

f_dir = input("2.파일을 저장할 폴더명만 쓰세요(기본경로 : c:\\py_temp\\): ")
if f_dir == '':
    f_dir = "c:\\py_temp\\"
else:
    f_dir = "c:\\py_temp\\" + f_dir
    
print("\n")

if cnt > 30:
    print("조금 오래 걸립니다")
else:
    print("조금만 기다려 주세요")
    
# 파일 경로와 이름을 지정
sec_name = '식품'
query_txt = '쿠팡'

n = time.localtime()
s1 = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)
name = f_dir + '-' +s1 + '-' + query_txt + '-' + sec_name

os.makedirs(name)
os.chdir(name)

name_txt = name + '\\' + s1 + '-' + query_txt + '-' + sec_name + '.txt'
name_csv = name + '\\' + s1 + '-' + query_txt + '-' + sec_name + '.csv'
name_xls = name + '\\' + s1 + '-' + query_txt + '-' + sec_name + '.xls'

# 이미지 저장용 폴더 생성
img_dir = name + "\\images"
os.makedirs(img_dir)
os.chdir(img_dir)

s_time = time.time()

# 웹사이트 접속 및 메뉴 이동
s = Service("C:/new/py_temp/chromedriver.exe")
driver = webdriver.Chrome(service=s)

query_url='https://www.coupang.com/'
driver.get(query_url)
driver.maximize_window()

driver.delete_all_cookies()
time.sleep(2)

driver.find_element(By.XPATH, '//*[@id="header"]/div').click()
driver.find_element(By.XPATH , '//*[@id="gnbAnalytics"]/ul[1]/li[4]/a').click( )


# 내용 수집하기
print("\n")
print("===== 곧 수집된 결과를 출력합니다 ^^ ===== ")
print("\n")

ranking2=[]        #제품의 판매순위 저장
title2=[]          #제품 정보 저장
p_price2=[]        #현재 판매가 저장
discount2 = []     #할인율 저장
sat_count2=[]      #상품평 수 저장

img_src2=[]   # 이미지 URL 저장변수
file_no = 0   # 이미지 파일 저장할 때 번호
count = 1     # 총 게시물 건수 카운트 변수

def scroll_down(driver):      
    driver.execute_script("window.scrollBy(0,1100);")
    time.sleep(4)
    
for x in range(1, page_cnt + 1):
    
    for cc in range(1, 7):
        scroll_down(driver)
     
        
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    item_result = soup.find('ul', 'baby-product-list').find_all('li')
    
    for li in item_result:
        if cnt < count :
            break
        try:
            photo = li.find('dt', 'image').find('img')['src']
        except AttributeError:
            continue
            
        file_no += 1
        full_photo = 'https:' + photo
        urllib.request.urlretrieve(full_photo, str(file_no) + '.png')
        time.sleep(0.1)
        
        f = open(name_txt, 'a', encoding='UTF-8')
        f.write("-----------------------------------------------------"+"\n")
        print("-" *70)
        
        ranking = count
        print("1.판매순위:",ranking)
        f.write('1.판매순위:'+ str(ranking) + "\n")

        try :
            t = li.find('div','name').get_text().replace("\n","")
        except :
            title = '제품소개가 없습니다'
            print(title.replace("\n",""))
            f.write('2.제품소개:'+ title + "\n")
        else :
            title = t.replace("\n","").strip()
            print("2.제품소개:", title.replace("\n","").strip())                  
            f.write('2.제품소개:'+ title + "\n")

        try :
            p_price = li.find('strong','price-value').get_text().replace("\n","")
        except :
            p_price = '0'
            print("3.판매가격:", p_price.replace("\n",""))
            f.write('3.판매가격:'+ p_price + "\n")
        else :
            print("3.판매가격:", p_price.replace("\n",""))
            f.write('3.판매가격:'+ p_price + "\n")

        try :
            discount = li.find('span','discount-percentage').get_text().replace("\n","")
        except  :
            discount = '0'
            print("4:할인률:", discount)
            f.write('4.할인율:'+ discount + "\n")
        else :
            print("4:할인률:", discount)
            f.write('4.할인율:'+ discount + "\n")

        try :
            sat_count_1 = li.find('span','rating-total-count').get_text()
            sat_count_2 = sat_count_1.replace("(","").replace(")","")
        except  :
            sat_count_2='0'
            print('5.상품평 수: ',sat_count_2)
            f.write('5.상품평 수:'+ sat_count_2 + "\n")
        else :
            print('5.상품평 수:',sat_count_2)
            f.write('5.상품평 수:'+ sat_count_2 + "\n")

        print("-" *70)

        f.close( )             
        time.sleep(0.5)

        ranking2.append(ranking)
        title2.append(title.replace("\n",""))

        p_price2.append(p_price.replace("\n",""))
        discount2.append(discount)

        try :   
            sat_count2.append(sat_count_2)
        except IndexError :
            sat_count2.append(0)

        count += 1
        
    x += 1       
    try :
        # # Access Denied 메시지가 나오면 아래코드로 쿠키를 삭제한다
        driver.delete_all_cookies()
        time.sleep(2)
        driver.find_element(By.LINK_TEXT, '%s' %x).click() # 다음 페이지번호 클릭
    except :
        break            
        
# # Access Denied 메시지가 나오면 아래코드로 쿠키를 삭제한다
driver.delete_all_cookies()
time.sleep(2)     

#step 6. csv , xls 형태로 저장하기              
co_best_seller = pd.DataFrame()
co_best_seller['판매순위']=ranking2
co_best_seller['제품소개']=pd.Series(title2)
co_best_seller['제품판매가']=pd.Series(p_price2)
co_best_seller['할인율']=pd.Series(discount2)
co_best_seller['상품평수']=pd.Series(sat_count2)
        
# csv 형태로 저장하기
co_best_seller.to_csv(name_csv, encoding="utf-8-sig", index=False)

co_best_seller.to_excel(name_xls, index=False, engine='openpyxl')

e_time = time.time()
t_time = e_time - s_time

count -= 1

print("=" *80)
print("1.요청된 총 %s 건의 리뷰 중에서 실제 크롤링 된 리뷰수는 %s 건입니다" %(cnt,count))
print("2.총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("3.파일 저장 완료: txt 파일명 : %s " %name_txt)
print("4.파일 저장 완료: csv 파일명 : %s " %name_csv)
print("5.파일 저장 완료: xls 파일명 : %s " %name_xls)
print("=" *80)
        
#Step 7. xls 파일에 제품 이미지 삽입하기
import win32com.client as win32   #pywin32 , pypiwin32 설치후 동작
import win32api                   #파이썬 프롬프트를 관리자 권한으로 실행해야 에러없음
        
excel = win32.Dispatch('Excel.Application')
wb = excel.Workbooks.Open(name_xls)
sheet = wb.ActiveSheet
sheet.Columns(2).ColumnWidth = 30   
row_cnt = cnt+1
sheet.Rows("2:%s" %row_cnt).RowHeight = 120       
 
ws = wb.Sheets("Sheet1")
col_name2=[]
file_name2=[]        
        
for a in range(2, cnt+2):
    col_name='B'+str(a)
    col_name2.append(col_name)
        
for b in range(1,cnt+1) :
    file_name=img_dir+'\\'+str(b)+'.jpg'
    file_name2.append(file_name)        
        
for i in range(0,cnt) :
    rng = ws.Range(col_name2[i])
    image = ws.Shapes.AddPicture(file_name2[i], False, True, rng.Left, rng.Top, 130, 100)
    excel.Visible=True
    excel.ActiveWorkbook.Save()        
        
        

********************************************************************************
쿠팡 사이트의 식품 카테고리 Best Seller 상품 정보 추출하기
********************************************************************************


KeyboardInterrupt: Interrupted by user